In [4]:
import numpy as np
import pandas as pd
import scipy.stats as sts
from random import randrange
import csv
import random
import math
import operator

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/OHrydko/Machine-Learning-Labs/master/lab2/MP-07-Hrydko.csv', 
                 header=None, names=['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'y_s'], sep=';',engine='python')

df.head()

,x0,x1,x2,x3,x4,x5,x6,y_s
0,F,yes,no,no,4,1,3,11
1,F,no,no,yes,5,1,3,11
2,F,yes,no,yes,4,3,3,12
3,F,no,yes,yes,3,1,5,14
4,F,no,no,no,4,2,5,13


In [6]:
df = df.replace({'yes': 1, 'no': 0})
df = pd.get_dummies(df)
df['y'] = df['y_s']
df = df.drop(['y_s'], axis=1)
df.head()

dataset = []
for index, row in df.iterrows():
    dataset.append(list(row))

In [11]:
import csv
import random
import math

def loadCsv(filename):
    lines = csv.reader(open(filename, "rb"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities
            
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.iteritems():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def main():
    splitRatio = 0.67
    print(dataset)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    # prepare model
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%').format(accuracy)

main()

[[1, 0, 0, 4, 1, 3, 1, 0, 11], [0, 0, 1, 5, 1, 3, 1, 0, 11], [1, 0, 1, 4, 3, 3, 1, 0, 12], [0, 1, 1, 3, 1, 5, 1, 0, 14], [0, 0, 0, 4, 2, 5, 1, 0, 13], [0, 1, 1, 5, 2, 5, 0, 1, 13], [0, 0, 1, 4, 1, 3, 0, 1, 13], [1, 0, 0, 4, 1, 1, 1, 0, 13], [0, 0, 1, 4, 1, 1, 0, 1, 17], [0, 1, 1, 5, 1, 5, 0, 1, 13], [0, 0, 1, 3, 2, 2, 1, 0, 14], [0, 1, 1, 5, 1, 4, 1, 0, 13], [0, 1, 1, 4, 3, 5, 0, 1, 12], [0, 0, 1, 5, 2, 3, 0, 1, 13], [0, 0, 1, 4, 1, 3, 0, 1, 15], [0, 0, 1, 4, 2, 2, 1, 0, 17], [0, 1, 1, 3, 2, 2, 1, 0, 14], [1, 1, 0, 5, 1, 4, 1, 0, 14], [0, 1, 1, 5, 4, 5, 0, 1, 7], [0, 1, 1, 3, 3, 5, 0, 1, 12], [0, 0, 1, 4, 1, 1, 0, 1, 14], [0, 0, 1, 5, 1, 5, 0, 1, 12], [0, 1, 1, 4, 3, 5, 0, 1, 14], [0, 1, 1, 5, 4, 5, 0, 1, 10], [1, 1, 1, 4, 1, 5, 1, 0, 10], [0, 0, 1, 1, 3, 5, 1, 0, 12], [0, 0, 1, 4, 2, 5, 0, 1, 12], [0, 0, 1, 2, 4, 1, 0, 1, 11], [1, 1, 1, 5, 1, 5, 0, 1, 13], [0, 1, 1, 4, 5, 5, 0, 1, 12], [0, 0, 1, 5, 4, 5, 0, 1, 11], [0, 1, 1, 4, 1, 5, 0, 1, 15], [0, 1, 1, 4, 1, 5, 0, 1, 15], [0, 1, 1, 

ZeroDivisionError: float division by zero